# 3er Entregable

Integrantes:
- Araoz, Tania
- Bajo, Pablo
- Barrera, Manuel

### Carga de librerias a utilizar 

In [1]:
import pandas as pd
from datetime import datetime
from scipy.sparse import csr_matrix
from lightfm import LightFM
import numpy as np

C:\Users\Manuel\AppData\Local\Temp\ipykernel_20396\790300137.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
c:\Users\Manuel\AppData\Local\Programs\Python\Python311\Lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


### Carga de datasets

In [2]:
movies = pd.read_csv("../data/ml-latest/movies.csv")
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [3]:
ratings = pd.read_csv("../data/ml-latest/ratings.csv")
ratings

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510
...,...,...,...,...
25000090,162541,50872,4.5,1240953372
25000091,162541,55768,2.5,1240951998
25000092,162541,56176,2.0,1240950697
25000093,162541,58559,4.0,1240953434


> Se usa el dataset de ratings para trabajar, tiene las interacciones entre usuarios y películas

In [4]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000095 entries, 0 to 25000094
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 762.9 MB


> El dataset contiene 100836 interacciones. <span style="color:red">ACTUALIZAR CON DATASET GRANDE</span>

> El timestamp está en formato int64, se debe convertir a formato fecha para poder trabajar.

In [5]:
ratings.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

> No hay valores nulos

In [6]:
ratings['userId'].nunique()

162541

> El dataset tiene 610 ususarios. <span style="color:red">Cambiar con dataset grande</span> 

In [7]:
ratings['movieId'].nunique()

59047

> el dataset contiene ratings de 9724 peliculas. <span style="color:red">Actualizar con dataset grande</span> 

In [8]:
ratings['rating'].sort_values(ascending=True).unique()

array([0.5, 1. , 1.5, 2. , 2.5, 3. , 3.5, 4. , 4.5, 5. ])

> Los valores posibles de ratings van del 0.5 al 5, con un incremento de 0.5. 

#### Preprocesado

Convertimos el timestamp numerico en formato fecha

In [9]:
ratings["timestamp"] = ratings["timestamp"].apply(lambda x: datetime.utcfromtimestamp(x).strftime('%Y/%m/%d'))

In [10]:
ratings["timestamp"]

0           2006/05/17
1           2006/05/17
2           2006/05/17
3           2006/05/17
4           2006/05/17
               ...    
25000090    2009/04/28
25000091    2009/04/28
25000092    2009/04/28
25000093    2009/04/28
25000094    2009/04/28
Name: timestamp, Length: 25000095, dtype: object

> Vemos que la fecha tiene un formato de fecha, pero la columna es de tipo object

Utilizando pandas convertimos a un formato de fechas que permita el filtrado

In [11]:
ratings["timestamp"] = pd.to_datetime(ratings['timestamp'], format='%Y/%m/%d')

In [12]:
ratings["timestamp"]

0          2006-05-17
1          2006-05-17
2          2006-05-17
3          2006-05-17
4          2006-05-17
              ...    
25000090   2009-04-28
25000091   2009-04-28
25000092   2009-04-28
25000093   2009-04-28
25000094   2009-04-28
Name: timestamp, Length: 25000095, dtype: datetime64[ns]

> Vemos que la columna tiene el formato datetime64

In [13]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,2006-05-17
1,1,306,3.5,2006-05-17
2,1,307,5.0,2006-05-17
3,1,665,5.0,2006-05-17
4,1,899,3.5,2006-05-17


Vemos el rango de fechas del dataset

In [14]:
ratings.timestamp.min()

Timestamp('1995-01-09 00:00:00')

In [15]:
ratings.timestamp.max()

Timestamp('2019-11-21 00:00:00')

> Vemos que el rango de fechas va desde el 29/03/1996 al 24/09/24

#### Dividimos dataset en train, test y validation
Vemos la catidad de ratings por año

In [16]:
plot_df = ratings.copy()
plot_df["year"] = ratings.timestamp.dt.year
plot_df = plot_df.groupby("year", as_index=False).count()[["year", "userId"]]
plot_df.columns = ["year", "reviews_count"]
plot_df.head(25)

,year,reviews_count
0,1995,3
1,1996,1430093
2,1997,626202
3,1998,272099
4,1999,1059080
5,2000,1735398
6,2001,1058750
7,2002,776654
8,2003,920295
9,2004,1048116


> Tomamos una proporción 80/20 para dividir el dataset en train - test

In [17]:
train = ratings[(ratings.timestamp < datetime(year=2015, month=1, day=1))]
train.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,2006-05-17
1,1,306,3.5,2006-05-17
2,1,307,5.0,2006-05-17
3,1,665,5.0,2006-05-17
4,1,899,3.5,2006-05-17


In [18]:
train.shape

(17436354, 4)

In [19]:
train.userId.nunique()

121673

In [20]:
train.movieId.nunique()

22316

In [21]:
test = ratings[ratings.timestamp >= datetime(year=2015, month=1, day=1)]
test.head()

,userId,movieId,rating,timestamp
254,3,1,4.0,2015-08-13
255,3,29,4.5,2017-01-18
256,3,32,4.5,2015-08-13
257,3,50,5.0,2015-08-13
258,3,111,4.0,2017-01-18


In [22]:
test.shape

(7563741, 4)

In [23]:
test.userId.nunique()

45450

In [24]:
test.movieId.nunique()

58133

In [25]:
plot_df = train.copy()
plot_df["year"] = train.timestamp.dt.year
plot_df = plot_df.groupby("year", as_index=False).count()[["year", "userId"]]
plot_df.columns = ["year", "reviews_count"]
plot_df.head(25)

,year,reviews_count
0,1995,3
1,1996,1430093
2,1997,626202
3,1998,272099
4,1999,1059080
5,2000,1735398
6,2001,1058750
7,2002,776654
8,2003,920295
9,2004,1048116


In [26]:
train.shape

(17436354, 4)

> Definimos el conjunto de validación, en función de nuestro conjunto de entrenamiento. <span style="color:red">Actualizar con dataset grande</span>

In [27]:
validation = train[train.timestamp >= datetime(year=2009, month=1, day=1)]
validation.head()

,userId,movieId,rating,timestamp
1740,12,101,4.0,2009-10-12
1863,12,908,4.5,2009-12-18
1868,12,919,4.0,2010-02-23
1870,12,926,4.5,2010-02-23
1889,12,1080,3.0,2009-12-18


In [28]:
validation.shape

(3908223, 4)

In [29]:
validation.userId.nunique()

30854

In [30]:
validation.movieId.nunique()

22075

In [31]:
plot_df = validation.copy()
plot_df["year"] = validation.timestamp.dt.year
plot_df = plot_df.groupby("year", as_index=False).count()[["year", "userId"]]
plot_df.columns = ["year", "reviews_count"]
plot_df.head(25)

,year,reviews_count
0,2009,810127
1,2010,792436
2,2011,676498
3,2012,635208
4,2013,515684
5,2014,478270


> Redefinimos el conjunto de entrenamiento. <span style="color:red">Actualizar con dataset grande</span>

In [32]:
train = train[(train.timestamp < datetime(year=2009, month=1, day=1))]
train.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,2006-05-17
1,1,306,3.5,2006-05-17
2,1,307,5.0,2006-05-17
3,1,665,5.0,2006-05-17
4,1,899,3.5,2006-05-17


In [33]:
plot_df = train.copy()
plot_df["year"] = train.timestamp.dt.year
plot_df = plot_df.groupby("year", as_index=False).count()[["year", "userId"]]
plot_df.columns = ["year", "reviews_count"]
plot_df.head(25)

,year,reviews_count
0,1995,3
1,1996,1430093
2,1997,626202
3,1998,272099
4,1999,1059080
5,2000,1735398
6,2001,1058750
7,2002,776654
8,2003,920295
9,2004,1048116


In [34]:
train.shape

(13528131, 4)

¿Tenemos COLDSTAR? 

In [35]:
test[~test.userId.isin(train.userId.unique())].userId.nunique()

43574

> Tenemos ### Usuarios que se encuentra en el dataset de test y no en el de train. <span style="color:red">Actualizar con dataset grande</span>

In [36]:
validation[~validation.userId.isin(train.userId.unique())].userId.nunique()

25796

> Tenemos ### Usuarios que se encuentra en el dataset de validation y no en el de train. <span style="color:red">Actualizar con dataset grande</span>

#### Matriz de Interacciones

In [37]:
interactions_train = train[["userId", "movieId", "rating"]].copy()
interactions_train.head()

,userId,movieId,rating
0,1,296,5.0
1,1,306,3.5
2,1,307,5.0
3,1,665,5.0
4,1,899,3.5


In [38]:
interactions_matrix = interactions_train.pivot(index="userId", columns="movieId", values="rating")

In [39]:
interactions_matrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,64969,64976,64983,64986,64990,64993,64997,65011,65025,65088
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
interactions_matrix = interactions_matrix.fillna(0)

In [41]:
interactions_matrix.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,64969,64976,64983,64986,64990,64993,64997,65011,65025,65088
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
interactions_matrix.shape

(95877, 10590)

In [43]:
interactions_matrix_csr = csr_matrix(interactions_matrix.values)

In [44]:
interactions_matrix_csr

<95877x10590 sparse matrix of type '<class 'numpy.float64'>'
	with 13528131 stored elements in Compressed Sparse Row format>

In [45]:
user_ids = list(interactions_matrix.index)
user_map = {}
counter = 0
for i in user_ids:
    user_map[i] = counter
    counter += 1

In [46]:
user_map

{1: 0,
 2: 1,
 5: 2,
 6: 3,
 7: 4,
 8: 5,
 9: 6,
 10: 7,
 11: 8,
 12: 9,
 15: 10,
 18: 11,
 20: 12,
 22: 13,
 23: 14,
 25: 15,
 27: 16,
 28: 17,
 32: 18,
 36: 19,
 41: 20,
 42: 21,
 43: 22,
 45: 23,
 47: 24,
 48: 25,
 49: 26,
 54: 27,
 56: 28,
 57: 29,
 59: 30,
 60: 31,
 61: 32,
 62: 33,
 66: 34,
 67: 35,
 72: 36,
 73: 37,
 74: 38,
 76: 39,
 77: 40,
 79: 41,
 80: 42,
 83: 43,
 85: 44,
 86: 45,
 90: 46,
 92: 47,
 94: 48,
 97: 49,
 100: 50,
 103: 51,
 105: 52,
 106: 53,
 107: 54,
 111: 55,
 112: 56,
 115: 57,
 116: 58,
 118: 59,
 120: 60,
 121: 61,
 122: 62,
 123: 63,
 124: 64,
 125: 65,
 129: 66,
 130: 67,
 131: 68,
 132: 69,
 133: 70,
 134: 71,
 135: 72,
 136: 73,
 137: 74,
 139: 75,
 140: 76,
 141: 77,
 143: 78,
 147: 79,
 152: 80,
 155: 81,
 156: 82,
 157: 83,
 158: 84,
 159: 85,
 160: 86,
 161: 87,
 162: 88,
 163: 89,
 164: 90,
 165: 91,
 166: 92,
 167: 93,
 168: 94,
 169: 95,
 171: 96,
 173: 97,
 175: 98,
 176: 99,
 177: 100,
 178: 101,
 179: 102,
 182: 103,
 183: 104,
 184: 105,
 

#### Modelo

In [47]:
%%time
model = LightFM(no_components=30, random_state=100, learning_rate=0.03, loss='logistic')

CPU times: total: 0 ns
Wall time: 9.53 ms


In [48]:
%%time
model = model.fit(interactions_matrix_csr, epochs=20)

CPU times: total: 5min 16s
Wall time: 20min 48s


In [49]:
model

#### Metodos auxiliares
Metodo para obtener las recomendaciones en caso de ColdStart

In [50]:
def getColdStarRecomm(dataset, no_recom): 
    '''
    Esta funcion recibe el dataset y retonar una lista de tamaño no_recom con recomendaciones mas populares 
    '''
    recomm = dataset.groupby("movieId", as_index=False).agg({"userId":"nunique"}).sort_values(by="userId", ascending=False)
    recomm.columns=["movieId", "popularity"]
    return recomm.movieId.values[:no_recom]

Metodo para obtener los scores a partir del modelo

In [51]:
def predict(user):
    '''
    Esta funcion recibe un usuario y retorna lista de scores
    '''
    user_predic = user_map[user]
    n_users, n_items = interactions_matrix.shape
    item_ids = np.arange(n_items)
    return model.predict(user_ids=user_predic, item_ids = item_ids)

Metodo para obtener las recomendaciones ordenadas

In [52]:
def getOrderedMoviesId(preds, no_recom):
    '''
    Esta funcion recibe los scores predecidos y retorna lista ordenada de moviesId
    '''
    recomm = pd.Series(preds)
    recomm.index = interactions_matrix.columns
    return list(pd.Series(recomm.sort_values(ascending=False).index))[:no_recom]

Metodo para eliminar de las recomendaciones las peliculas ya vistas

In [53]:
def getNotWatchedMovieId(user, recomm):
    '''
    Esta funcion recibe las recomendaciones y retorna lista de recomendaciones sin las peliculas vistas por el usuario
    '''
    watched = train[train.userId == user].movieId.unique()
    return [x for x in recomm if x not in watched][:20]

In [54]:
def recomm(user, no_recom):
    result = None
    if user in list(interactions_matrix.index):
        score = predict(user)
        recomm = getOrderedMoviesId(score, no_recom)
        result = getNotWatchedMovieId(user, recomm)
    else:
        result = getColdStarRecomm(train, 20)
    return result

#### Recomendaciones
Generamos metricas para todos los ususarios de validation

In [55]:
def recommAll_Validation(no_recom):
    recomms_dict = {
        'user_id': [],
        'recomms': []
    }
    for user in validation.userId.unique():
        recomms_dict['user_id'].append(user)
        recomms_dict['recomms'].append(recomm(user, no_recom))
    return pd.DataFrame(recomms_dict)


In [ ]:
pd.options.display.max_colwidth = None
df = recommAll_Validation(100)
df

In [ ]:
df.columns

Index(['user_id', 'recomms'], dtype='object')

> Como de los ## usuarios del dataset validation, ## son usuarios nuevos, les asigna las recomendaciones coldstart <span style="color:red">Actualizar con dataset grande</span>

In [ ]:
df.user_id.nunique()

69

#### Comparación

> Primero generamos la recomendaciones ideales del conjunto de validación.

In [ ]:
interactions_validation = validation[["userId", "movieId", "rating"]].copy()
interactions_validation.head()

,userId,movieId,rating
232,2,318,3.0
233,2,333,4.0
234,2,1704,4.5
235,2,3578,4.0
236,2,6874,4.0


In [ ]:
interactions_validation.columns

Index(['userId', 'movieId', 'rating'], dtype='object')

In [ ]:
ideal_recomms = interactions_validation.sort_values(by=["userId", "rating"], ascending=False)\
                  .groupby("userId", as_index=False)\
                  .agg({"movieId": "unique"})
ideal_recomms

,userId,movieId
0,2,"[60756, 80906, 89774, 106782, 122882, 131724, 1704, 58559, 68157, 80489, 333, 3578, 6874, 46970, 48516, 74458, 79132, 86345, 112552, 8798, 91529, 99114, 115713, 318, 71535, 77455, 109487, 91658, 114060]"
1,21,"[10, 1270, 2011, 2012, 7573, 260, 356, 648, 1196, 1210, 1544, 1580, 2947, 2948, 2949, 2989, 2990, 2991, 2993, 3633, 3635, 3638, 3639, 3984, 4489, 4963, 5445, 7569, 7570, 8529, 8984, 33004, 33493, 53121, 53322, 58998, 68954, 78637, 101864, 111759, 111781, 135887, 364, 480, 588, 597, 743, 1198, 1291, 2115, 2424, 2529, 2571, 2628, 2671, 2763, 2916, 3022, 3253, 3868, 3869, 4005, 4306, 4545, 4896, 5218, 5378, 5418, 5574, 5816, 6539, 6934, 6942, 7143, 8360, 8368, 8644, 8665, 8798, 8972, 33615, 40815, 47566, 49272, 53125, 54286, 59315, 59615, 69644, 69844, 72998, 77561, 78499, 79185, 82202, 85259, 88125, 89745, 91630, 94677, ...]"
2,29,"[1408, 5464, 6502, 111362, 4223, 5010, 104841, 111759]"
3,60,"[527, 858, 58559, 318, 362, 783, 805, 1242, 2150, 2739, 3386, 3424, 6016, 48, 50, 60, 455, 832, 1203, 1562, 2067, 2724]"
4,63,"[1, 50, 260, 296, 318, 344, 745, 1080, 1136, 1148, 1196, 1198, 1208, 1220, 1223, 1270, 1288, 2078, 2716, 2788, 2858, 2959, 3949, 5669, 6104, 7361, 8874, 32587, 33779, 38038, 48774, 51255, 57669, 58559, 77455, 77800, 79702, 86290, 89753, 89904, 91529, 92494, 92535, 97913, 98491, 102217, 106696, 108932, 111781, 115617, 134853, 608, 1210, 1965, 2542, 2692, 6016, 6350, 7980, 48516, 64285, 81845, 96610, 102445, 47, 165, 357, 364, 367, 527, 588, 589, 592, 648, 778, 858, 912, 913, 953, 1036, 1084, 1193, 1201, 1202, 1203, 1213, 1221, 1262, 1282, 1291, 1617, 1968, 2115, 2329, 2791, 3088, 3435, 3481, 3535, 3751, ...]"
...,...,...
64,573,"[858, 109487, 111362, 112852]"
65,581,"[318, 527, 3147, 4896, 5816, 5995, 7361, 356, 2324, 2762, 2959, 3949, 4022, 4226, 4306, 4886, 4993, 4995, 5349, 5952, 6377, 7153, 8368, 44191, 79132, 81845, 92259, 109487, 112552, 1704, 59315, 60069, 68954, 116797, 134130, 2571, 4973, 5989, 7147, 48394]"
66,582,"[2571, 79091, 79132, 81834, 88125, 89745, 92259, 49272, 58559, 69844, 76093, 91529, 96079, 99114, 102125, 104841, 109487, 260, 1196, 48516, 54001, 60069, 68157, 74458, 81229, 81564, 87232, 91500, 91630, 94864, 109374, 134130, 4993, 5618, 44191, 48780, 68954, 73321, 115617, 84954, 97752, 97913, 76251, 77561, 85414, 96610]"
67,598,"[5816, 7361, 46578, 54001, 56367, 79132, 101577, 103543, 113829, 114265, 124851, 130490, 4963, 5418, 63082, 1197, 110771, 4226, 4306, 8360, 593]"


In [ ]:
ideal_recomms.columns

Index(['userId', 'movieId'], dtype='object')

> Proximo paso sería hacer un nuevo dataframe, en donde combinariamos lo recomendado con la lista ideal para poder hacer la comparación.

In [ ]:
merged_df = pd.merge(df, ideal_recomms, left_on='user_id', right_on='userId', how='inner')
merged_df = merged_df.rename(columns={'recomms': 'recomms_df', 'movieId': 'recomms_ideal'})
merged_df = merged_df.drop(columns=['userId'])

merged_df



,user_id,recomms_df,recomms_ideal
0,2,"[296, 356, 318, 593, 480, 110, 589, 780, 260, 150, 2571, 592, 1, 457, 380, 527, 590, 50, 47, 2858]","[60756, 80906, 89774, 106782, 122882, 131724, 1704, 58559, 68157, 80489, 333, 3578, 6874, 46970, 48516, 74458, 79132, 86345, 112552, 8798, 91529, 99114, 115713, 318, 71535, 77455, 109487, 91658, 114060]"
1,21,"[296, 356, 318, 110, 593, 480, 150, 592, 590, 380, 589, 457, 344, 780, 588, 153, 1, 527, 47, 377]","[10, 1270, 2011, 2012, 7573, 260, 356, 648, 1196, 1210, 1544, 1580, 2947, 2948, 2949, 2989, 2990, 2991, 2993, 3633, 3635, 3638, 3639, 3984, 4489, 4963, 5445, 7569, 7570, 8529, 8984, 33004, 33493, 53121, 53322, 58998, 68954, 78637, 101864, 111759, 111781, 135887, 364, 480, 588, 597, 743, 1198, 1291, 2115, 2424, 2529, 2571, 2628, 2671, 2763, 2916, 3022, 3253, 3868, 3869, 4005, 4306, 4545, 4896, 5218, 5378, 5418, 5574, 5816, 6539, 6934, 6942, 7143, 8360, 8368, 8644, 8665, 8798, 8972, 33615, 40815, 47566, 49272, 53125, 54286, 59315, 59615, 69644, 69844, 72998, 77561, 78499, 79185, 82202, 85259, 88125, 89745, 91630, 94677, ...]"
2,29,"[110, 593, 480, 592, 590, 380, 457, 589, 344, 588, 153, 1, 527, 32, 47, 377, 349, 260, 231, 595]","[1408, 5464, 6502, 111362, 4223, 5010, 104841, 111759]"
3,60,"[296, 356, 318, 593, 480, 110, 589, 780, 260, 150, 2571, 592, 1, 457, 380, 527, 590, 50, 47, 2858]","[527, 858, 58559, 318, 362, 783, 805, 1242, 2150, 2739, 3386, 3424, 6016, 48, 50, 60, 455, 832, 1203, 1562, 2067, 2724]"
4,63,"[296, 356, 318, 593, 480, 110, 589, 780, 260, 150, 2571, 592, 1, 457, 380, 527, 590, 50, 47, 2858]","[1, 50, 260, 296, 318, 344, 745, 1080, 1136, 1148, 1196, 1198, 1208, 1220, 1223, 1270, 1288, 2078, 2716, 2788, 2858, 2959, 3949, 5669, 6104, 7361, 8874, 32587, 33779, 38038, 48774, 51255, 57669, 58559, 77455, 77800, 79702, 86290, 89753, 89904, 91529, 92494, 92535, 97913, 98491, 102217, 106696, 108932, 111781, 115617, 134853, 608, 1210, 1965, 2542, 2692, 6016, 6350, 7980, 48516, 64285, 81845, 96610, 102445, 47, 165, 357, 364, 367, 527, 588, 589, 592, 648, 778, 858, 912, 913, 953, 1036, 1084, 1193, 1201, 1202, 1203, 1213, 1221, 1262, 1282, 1291, 1617, 1968, 2115, 2329, 2791, 3088, 3435, 3481, 3535, 3751, ...]"
...,...,...,...
64,573,"[590, 344, 527, 32, 349, 161, 434, 292, 208, 454, 329, 253, 608, 858, 34, 339, 597, 300, 587, 539]","[858, 109487, 111362, 112852]"
65,581,"[296, 356, 318, 593, 480, 110, 589, 780, 260, 150, 2571, 592, 1, 457, 380, 527, 590, 50, 47, 2858]","[318, 527, 3147, 4896, 5816, 5995, 7361, 356, 2324, 2762, 2959, 3949, 4022, 4226, 4306, 4886, 4993, 4995, 5349, 5952, 6377, 7153, 8368, 44191, 79132, 81845, 92259, 109487, 112552, 1704, 59315, 60069, 68954, 116797, 134130, 2571, 4973, 5989, 7147, 48394]"
66,582,"[296, 356, 318, 593, 480, 110, 589, 780, 260, 150, 2571, 592, 1, 457, 380, 527, 590, 50, 47, 2858]","[2571, 79091, 79132, 81834, 88125, 89745, 92259, 49272, 58559, 69844, 76093, 91529, 96079, 99114, 102125, 104841, 109487, 260, 1196, 48516, 54001, 60069, 68157, 74458, 81229, 81564, 87232, 91500, 91630, 94864, 109374, 134130, 4993, 5618, 44191, 48780, 68954, 73321, 115617, 84954, 97752, 97913, 76251, 77561, 85414, 96610]"
67,598,"[296, 356, 318, 593, 480, 110, 589, 780, 260, 150, 2571, 592, 1, 457, 380, 527, 590, 50, 47, 2858]","[5816, 7361, 46578, 54001, 56367, 79132, 101577, 103543, 113829, 114265, 124851, 130490, 4963, 5418, 63082, 1197, 110771, 4226, 4306, 8360, 593]"


In [ ]:
aps = []

for pred, label in merged_df[["recomms_ideal", "recomms_df"]].values:
  n = len(pred)
  arange = np.arange(n, dtype=np.int32) + 1.
  rel_k = np.in1d(pred[:n], label)
  tp = np.ones(rel_k.sum(), dtype=np.int32).cumsum()
  denom = arange[rel_k]
  ap = (tp / denom).sum() / len(label)
  aps.append(ap)

In [ ]:
MAP = np.mean(aps)
print(f'mean average precision = {round(MAP, 5)}')

mean average precision = 0.08557


#### Metricas en test

In [ ]:
def recommAll_test(no_recom):
    recomms_dict = {
        'user_id': [],
        'recomms': []
    }
    for user in test.userId.unique():
        recomms_dict['user_id'].append(user)
        recomms_dict['recomms'].append(recomm(user, no_recom))
    return pd.DataFrame(recomms_dict)

In [ ]:
pd.options.display.max_colwidth = None
df_test = recommAll_test(100)
df_test

,user_id,recomms
0,10,"[296, 356, 318, 593, 480, 110, 589, 780, 260, 150, 2571, 592, 1, 457, 380, 527, 590, 50, 47, 2858]"
1,15,"[296, 356, 318, 110, 593, 480, 150, 592, 380, 590, 589, 457, 344, 780, 588, 1, 153, 527, 32, 47]"
2,18,"[296, 356, 318, 593, 480, 110, 589, 780, 260, 150, 2571, 592, 1, 457, 380, 527, 590, 50, 47, 2858]"
3,21,"[296, 356, 318, 110, 593, 480, 150, 592, 590, 380, 589, 457, 344, 780, 588, 153, 1, 527, 47, 377]"
4,24,"[296, 356, 318, 593, 480, 110, 589, 780, 260, 150, 2571, 592, 1, 457, 380, 527, 590, 50, 47, 2858]"
...,...,...
115,586,"[296, 356, 318, 593, 480, 110, 589, 780, 260, 150, 2571, 592, 1, 457, 380, 527, 590, 50, 47, 2858]"
116,596,"[296, 356, 318, 593, 480, 110, 589, 780, 260, 150, 2571, 592, 1, 457, 380, 527, 590, 50, 47, 2858]"
117,599,"[296, 356, 318, 593, 480, 110, 589, 780, 260, 150, 2571, 592, 1, 457, 380, 527, 590, 50, 47, 2858]"
118,601,"[296, 356, 318, 593, 480, 110, 589, 780, 260, 150, 2571, 592, 1, 457, 380, 527, 590, 50, 47, 2858]"


In [ ]:
interactions_test = test[["userId", "movieId", "rating"]].copy()
interactions_test.head()

,userId,movieId,rating
1119,10,296,1.0
1120,10,356,3.5
1121,10,588,4.0
1122,10,597,3.5
1123,10,912,4.0


In [ ]:
ideal_recomms2 = interactions_test.sort_values(by=["userId", "rating"], ascending=False)\
                  .groupby("userId", as_index=False)\
                  .agg({"movieId": "unique"})
ideal_recomms2

,userId,movieId
0,10,"[7458, 8533, 8869, 33794, 49272, 49286, 71579, 79091, 81845, 91529, 92259, 96079, 136020, 140110, 4306, 4447, 7169, 31685, 51705, 58559, 63992, 69406, 94070, 106696, 113275, 588, 912, 1907, 3578, 4993, 4995, 5952, 6535, 6942, 7149, 7153, 7154, 7375, 40819, 68954, 88163, 95167, 95449, 103335, 103339, 104374, 109853, 112006, 113394, 137595, 356, 597, 1784, 2671, 4246, 5377, 6377, 7293, 7451, 8529, 8636, 8665, 8969, 30749, 54286, 56367, 58047, 63113, 66203, 72330, 72720, 72737, 80549, 81847, 82167, 84374, 87222, 95543, 106489, 129428, 1088, 1247, 1307, 3882, 5066, 5620, 5943, 5957, 6155, 6266, 7151, 8808, 33145, 33679, 40629, 47099, 51662, 56949, 60397, 69844, ...]"
1,15,"[260, 318, 356, 527, 589, 1196, 1200, 1210, 1214, 1270, 2011, 3147, 3156, 3578, 4720, 4995, 5989, 33493, 84152, 122886, 152077, 166528, 1653, 2329, 2916, 48304, 48780, 84954, 104841, 111759, 112556, 134853, 296, 858, 1198, 1240, 2012, 2571, 2858, 3499, 3949, 4370, 5445, 64614, 71057, 97938, 101864, 105504, 109487, 134130, 47, 780, 1265, 2028, 3535, 4022, 4886, 4993, 5952, 6502, 7254, 8644, 56174, 60069, 68954, 70286, 79132, 85414, 103249, 158872, 160980, 166635, 293, 364, 588, 1527, 2081, 2762, 3753, 3994, 4306, 5618, 6377, 6874, 7438, 48774, 50872, 63859, 68237, 72998, 91500, 94864, 96610, 143385, 152081, 1, 2959, 8360, 8961, 71264, ...]"
2,18,"[50, 318, 923, 1201, 1203, 1209, 1221, 16, 47, 110, 235, 293, 356, 527, 589, 593, 608, 778, 904, 1080, 1136, 1148, 1193, 1206, 1207, 1210, 1212, 1213, 1219, 1222, 1223, 1227, 1234, 1247, 1356, 1374, 1732, 2324, 2542, 2571, 2762, 2951, 2959, 3052, 3275, 3578, 3681, 3949, 4011, 4226, 4993, 4995, 5008, 5120, 5995, 6300, 6440, 6539, 6807, 7147, 7153, 7254, 7843, 27716, 27878, 33794, 44191, 44199, 44665, 48516, 48780, 51540, 52604, 55118, 55290, 55765, 64197, 67255, 68073, 71108, 71899, 73017, 73323, 74458, 74510, 76251, 79132, 81788, 84392, 109487, 112334, 112552, 112852, 115713, 116797, 134130, 142488, 157108, 157110, 177593, ...]"
3,21,"[47997, 2717, 33679, 117529, 119145, 122886, 122896, 122922, 136020, 143385, 152081, 164179, 167036, 296, 541, 780, 2617, 6155, 30793, 32296, 34048, 36519, 50872, 51662, 53996, 60397, 63082, 68791, 69122, 69526, 72378, 73321, 78469, 79293, 79592, 87520, 90249, 91535, 95167, 96588, 108190, 112138, 114180, 115149, 116823, 122900, 122904, 135133, 135536, 138036, 164909, 166492, 168248, 902, 2052, 2953, 4700, 5219, 5254, 7373, 32587, 34150, 41566, 72641, 93510, 110553, 126548, 130450, 136016, 168252, 1573, 3697, 142536, 58025, 97913, 148675, 6874, 7438, 38038, 143245, 149380, 2174, 5266, 61160, 108932, 160565, 1391, 160872, 173307]"
4,24,"[6, 318, 356, 593, 1198, 1265, 3147, 5064, 6350, 27773, 50, 296, 608, 1197, 1246, 1396, 1527, 1580, 1704, 1784, 2028, 2115, 2424, 2571, 2686, 3578, 4027, 4262, 4489, 4855, 5418, 5673, 5791, 7143, 35836, 38061, 44191, 46976, 51662, 54286, 58559, 58998, 64957, 68358, 70286, 72011, 79132, 86882, 91529, 94777, 119145, 132660, 134130, 134853, 32, 165, 253, 316, 457, 552, 780, 1220, 1370, 1663, 1682, 2273, 2421, 2617, 2916, 4299, 4701, 4973, 4995, 7293, 31685, 33679, 34437, 49272, 52973, 57368, 59615, 61024, 91542, 96079, 102407, 111759, 122886, 733, 1297, 1639, 1653, 2134, 5445, 5903, 8784, 31696, 45672, 47610, 61132, 63113, ...]"
...,...,...
115,586,"[110, 318, 589, 1198, 1200, 1374, 1580, 1704, 2011, 2353, 2490, 2571, 3175, 3578, 3753, 3793, 4886, 4993, 5952, 6333, 6539, 7153, 8368, 8665, 33615, 45431, 45499, 45517, 47610, 50872, 59315, 59369, 59784, 60069, 62999, 63859, 68954, 76093, 77561, 79091, 86298, 86880, 87222, 93272, 95167, 96861, 98243, 101142, 103141, 106489, 106696, 110102, 112852, 117851, 118696, 120635, 122886, 122896, 122906, 122918, 122920, 122922, 122926, 134853, 135133, 136556, 149406, 152081, 160438, 166461, 168252, 168418, 179819, 187595, 161, 260, 380, 457, 553, 588, 1073, 1196, 1210, 1265, 1270, 1376, 1610, 3114, 5459, 27619, 33493, 41566, 42738, 52287, 54001, 54286, 54648, 58559, 65682, 78499, ...]"
116,596,"[2288, 300

In [ ]:
merged_df_test = pd.merge(df, ideal_recomms2, left_on='user_id', right_on='userId', how='inner')
merged_df_test = merged_df_test.rename(columns={'recomms': 'recomms_df', 'movieId': 'recomms_ideal'})
merged_df_test = merged_df_test.drop(columns=['userId'])

merged_df_test

,user_id,recomms_df,recomms_ideal
0,21,"[296, 356, 318, 110, 593, 480, 150, 592, 590, 380, 589, 457, 344, 780, 588, 153, 1, 527, 47, 377]","[47997, 2717, 33679, 117529, 119145, 122886, 122896, 122922, 136020, 143385, 152081, 164179, 167036, 296, 541, 780, 2617, 6155, 30793, 32296, 34048, 36519, 50872, 51662, 53996, 60397, 63082, 68791, 69122, 69526, 72378, 73321, 78469, 79293, 79592, 87520, 90249, 91535, 95167, 96588, 108190, 112138, 114180, 115149, 116823, 122900, 122904, 135133, 135536, 138036, 164909, 166492, 168248, 902, 2052, 2953, 4700, 5219, 5254, 7373, 32587, 34150, 41566, 72641, 93510, 110553, 126548, 130450, 136016, 168252, 1573, 3697, 142536, 58025, 97913, 148675, 6874, 7438, 38038, 143245, 149380, 2174, 5266, 61160, 108932, 160565, 1391, 160872, 173307]"
1,103,"[296, 356, 318, 593, 480, 110, 589, 780, 260, 150, 2571, 592, 1, 457, 380, 527, 590, 50, 47, 2858]","[1206, 1222, 2997, 3160, 64614, 81932, 148626, 48516, 56782, 86882, 923, 1945, 4677, 6873, 61323, 69140, 96728, 116799, 367, 2018, 2421, 4254, 7367, 50912, 106766, 168250, 122886]"
2,105,"[296, 356, 318, 593, 480, 110, 589, 780, 260, 150, 2571, 592, 1, 457, 380, 527, 590, 50, 47, 2858]","[2131, 4788, 5889, 37731, 57183, 80124, 104780, 116897, 120138, 120478, 129514, 130970, 134095, 136445, 136447, 138835, 140265, 141816, 142020, 143511, 145994, 147196, 147250, 147286, 147300, 147326, 147328, 147330, 150548, 151769, 159811, 159817, 163072, 163112, 163386, 163925, 165959, 166183, 170597, 170705, 170777, 171011, 171749, 172577, 172583, 172585, 172587, 172589, 172637, 172793, 172909, 173351, 173355, 173619, 173963, 174551, 175293, 175387, 175397, 175431, 179133, 56060, 86345, 86347, 92535, 98604, 104069, 107408, 120625, 127098, 133712, 133716, 134326, 140038, 141818, 141830, 152081, 163134, 163645, 172825, 173535, 175435, 935, 75341, 101088, 118888, 118896, 122904, 141810, 141820, 147282, 152591, 170411, 175401, 178613, 8533, 54997, 158402]"
3,112,"[296, 356, 318, 593, 480, 110, 589, 780, 260, 150, 2571, 592, 1, 457, 380, 527, 590, 50, 47, 2858]","[91529, 6, 16, 253, 508, 68157, 79132, 19, 62, 104, 141, 288, 292, 454, 1356, 1393, 3717, 68954, 72998, 99114, 551, 778, 70286, 25, 208, 329, 21, 185, 2, 95, 161, 223, 339, 788, 17, 36, 39, 300, 357, 434]"
4,119,"[296, 356, 318, 593, 480, 110, 589, 780, 260, 150, 2571, 592, 1, 457, 380, 527, 590, 50, 47, 2858]","[67255, 103984, 136598, 138210, 148626, 318, 7458, 56174, 89087, 112552, 122904, 131023, 135532, 141004, 142488, 157699, 162350, 165551, 168252, 48774, 50872, 108689, 110718, 110730, 114795, 117533, 129354, 134853, 139385, 139642, 166461, 7454, 8464, 8622, 51077, 64839, 136562, 140711]"
5,233,"[296, 356, 318, 593, 480, 110, 589, 780, 260, 150, 2571, 592, 1, 457, 380, 527, 590, 50, 47, 2858]","[356, 177593, 2571, 3949, 58559, 50, 608, 858, 1206, 1298, 2028, 2324, 2997, 3081, 5418, 5952, 5954, 5995, 6377, 7371, 48516, 60069, 74458, 106100, 112183, 159817, 170705, 47, 364, 778, 924, 1059, 1721, 1921, 2692, 2762, 4239, 4262, 4995, 5816, 5956, 5989, 6016, 6874, 7438, 8368, 8961, 33794, 44555, 49272, 73017, 79132, 81591, 81845, 91529, 92259, 99114, 168266, 180031, 183897, 1, 110, 648, 1136, 1193, 1225, 1580, 1884, 2858, 3285, 6365, 6539, 7458, 27803, 32587, 44191, 54286, 55765, 55872, 57669, 58025, 59315, 69122, 69134, 70286, 86298, 91658, 96020, 109487, 122904, 134214, 178061, 780, 1089, 3948, 6863, 6934, 51086, 72998, 78103, ...]"
6,249,"[592, 590, 153, 165, 316, 349, 231, 161, 292, 454, 329, 185, 10, 34, 339, 597, 300, 587, 539, 39]","[96821, 139385, 164179, 166528, 168252, 187593, 1193, 7438, 122904, 122906, 122912, 122916, 122918, 122920, 122922, 122926, 128360, 137857, 142488, 143355, 148626, 152081, 152970, 158238, 158872, 160718, 166534, 166643, 171763, 175569, 176371, 179819, 185029, 541, 4896, 7254, 8914, 81834, 88125, 108729, 115149, 116823, 122890, 122924, 135133, 135569, 140715, 152077, 157296, 159093, 159755, 161131, 167746, 168254, 168266, 169984, 176101, 182715, 184253, 2717, 120799, 142422, 149352, 16

In [ ]:
aps = []

for pred, label in merged_df_test[["recomms_ideal","recomms_df"]].values:
  n = len(pred)
  arange = np.arange(n, dtype=np.int32) + 1.
  rel_k = np.in1d(pred[:n], label)
  tp = np.ones(rel_k.sum(), dtype=np.int32).cumsum()
  denom = arange[rel_k]
  ap = (tp / denom).sum() / len(label)
  aps.append(ap)

In [ ]:
MAP = np.mean(aps)
print(f'mean average precision = {round(MAP, 5)}')

mean average precision = 0.02377
